In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import argparse
import torch
import sys

/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/agostini/miniconda/envs/dles/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/agostini/miniconda/envs/dles/lib/python3

In [3]:
import tensorflow as tf

In [4]:
args ={}
args['syn_model_dir']=Path('synthesizer/saved_models/logs-pretrained')
args['low_mem']=False

In [5]:
## Print some environment information (for debugging purposes)
print("Running a test of your configuration...\n")
if not torch.cuda.is_available():
    print("Your PyTorch installation is not configured to use CUDA. If you have a GPU ready "
          "for deep learning, ensure that the drivers are properly installed, and that your "
          "CUDA version matches your PyTorch installation. CPU-only inference is currently "
          "not supported.", file=sys.stderr)
    quit(-1)
device_id = torch.cuda.current_device()
gpu_properties = torch.cuda.get_device_properties(device_id)
print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
      "%.1fGb total memory.\n" % 
      (torch.cuda.device_count(),
       device_id,
       gpu_properties.name,
       gpu_properties.major,
       gpu_properties.minor,
       gpu_properties.total_memory / 1e9))

Running a test of your configuration...

Found 1 GPUs available. Using GPU 0 (GeForce RTX 2080 Ti) of compute capability 7.5 with 11.5Gb total memory.



In [6]:
## Load the models one by one.
print("Preparing the encoder, the synthesizer and the vocoder...")
synthesizer = Synthesizer(args['syn_model_dir'].joinpath("taco_pretrained"), low_mem=args['low_mem'])

Preparing the encoder, the synthesizer and the vocoder...
Found synthesizer "pretrained" trained to step 278000


In [7]:
## Run a test
print("Testing your configuration with small inputs.")
# Create a dummy embedding. You would normally use the embedding that encoder.embed_utterance
# returns, but here we're going to make one ourselves just for the sake of showing that it's
# possible.
embed = np.random.rand(speaker_embedding_size)
# Embeddings are L2-normalized (this isn't important here, but if you want to make your own 
# embeddings it will be).
embed /= np.linalg.norm(embed)
# The synthesizer can handle multiple inputs with batching. Let's create another embedding to 
# illustrate that
embeds = [embed, np.zeros(speaker_embedding_size)]
texts = ["test 1", "test 2"]
print("\tTesting the synthesizer... (loading the model will output a lot of text)")
mels = synthesizer.synthesize_spectrograms(texts, embeds)

print("All test passed! You can now synthesize speech.\n\n")

Testing your configuration with small inputs.
	Testing the synthesizer... (loading the model will output a lot of text)

Constructing model: Tacotron


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This class is equivalent as 

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use keras.layers.dense instead.


initialisation done /gpu:0

Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?)
  device:                   0
  embedding:                (?, ?, 512)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       28.439 Million.
Loading checkpoint: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000
All test passed! You can now synthesi

If you were not paying attention, the cell above print thw following parameters:
```
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?)
  device:                   0
  embedding:                (?, ?, 512)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       28.439 Million.
```

In [8]:
%%timeit
# Hit shift enter to run it again
mels = synthesizer.synthesize_spectrograms(texts, embeds)

593 ms ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
synthesizer._model.model.all_vars

[<tf.Variable 'Tacotron_model/inference/inputs_embedding:0' shape=(66, 512) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/kernel:0' shape=(5, 512, 512) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/kernel:0' shape=(5, 512, 512) dtype=float32_ref>,
 <tf.Variable 'Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/bias:0' shape=(512,) dtype=float32_ref>,
 

In [10]:
for var in synthesizer._model.model.all_vars:
    print('{:15} {} {}'.format(
        '{}'.format(var.get_shape().as_list()),
        var.dtype,
        var.name))

[66, 512]       <dtype: 'float32_ref'> Tacotron_model/inference/inputs_embedding:0
[5, 512, 512]   <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/kernel:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/bias:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/gamma:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/beta:0
[5, 512, 512]   <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/kernel:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/bias:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/con

In [11]:
# Number of total trainable params
n_of_trainable = int(np.sum([np.prod(v.shape) for v in tf.trainable_variables()]))
pre_quantization_size = n_of_trainable*4/1024/1024

print('Number of total trainable params: {}'.format(n_of_trainable))
print('With float32, 4 bytes per parameter: {} Bytes'.format(n_of_trainable*4))
print('Which is equivalent to: {:.3f} MB'.format(pre_quantization_size))

Number of total trainable params: 28439026
With float32, 4 bytes per parameter: 113756104 Bytes
Which is equivalent to: 108.486 MB


# Quantization experiment

In [85]:
with tf.Session() as sess:
    for var in synthesizer._model.model.all_vars:
        var = tf.quantization.quantize(
            var, -1,1,tf.qint8, mode='MIN_COMBINED',
            round_mode='HALF_AWAY_FROM_ZERO', name=None
        )
        print(var)

QuantizeV2(output=<tf.Tensor 'QuantizeV2_116:0' shape=(66, 512) dtype=qint8>, output_min=<tf.Tensor 'QuantizeV2_116:1' shape=() dtype=float32>, output_max=<tf.Tensor 'QuantizeV2_116:2' shape=() dtype=float32>)
QuantizeV2(output=<tf.Tensor 'QuantizeV2_117:0' shape=(5, 512, 512) dtype=qint8>, output_min=<tf.Tensor 'QuantizeV2_117:1' shape=() dtype=float32>, output_max=<tf.Tensor 'QuantizeV2_117:2' shape=() dtype=float32>)
QuantizeV2(output=<tf.Tensor 'QuantizeV2_118:0' shape=(512,) dtype=qint8>, output_min=<tf.Tensor 'QuantizeV2_118:1' shape=() dtype=float32>, output_max=<tf.Tensor 'QuantizeV2_118:2' shape=() dtype=float32>)
QuantizeV2(output=<tf.Tensor 'QuantizeV2_119:0' shape=(512,) dtype=qint8>, output_min=<tf.Tensor 'QuantizeV2_119:1' shape=() dtype=float32>, output_max=<tf.Tensor 'QuantizeV2_119:2' shape=() dtype=float32>)
QuantizeV2(output=<tf.Tensor 'QuantizeV2_120:0' shape=(512,) dtype=qint8>, output_min=<tf.Tensor 'QuantizeV2_120:1' shape=() dtype=float32>, output_max=<tf.Tensor

In [86]:
for var in synthesizer._model.model.all_vars:
    print('{:15} {} {}'.format(
        '{}'.format(var.get_shape().as_list()),
        var.dtype,
        var.name))

[66, 512]       <dtype: 'float32_ref'> Tacotron_model/inference/inputs_embedding:0
[5, 512, 512]   <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/kernel:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/conv1d/bias:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/gamma:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_1_encoder_convolutions/batch_normalization/beta:0
[5, 512, 512]   <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/kernel:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/conv_layer_2_encoder_convolutions/conv1d/bias:0
[512]           <dtype: 'float32_ref'> Tacotron_model/inference/encoder_convolutions/con

In [17]:
g = tf.get_default_graph()
tf.contrib.quantize.create_eval_graph(input_graph=g)

INFO:tensorflow:Cannot use 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/Relu' as input to 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/act_quant/FakeQuantWithMinMaxVars' because 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/Relu' is in a while loop.

Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/act_quant/FakeQuantWithMinMaxVars while context: None
Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/Relu while context: Tacotron_model/inference/decoder/while/while_context

Traceback for Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/act_quant/FakeQuantWithMinMaxVars:
  File "/home/agostini/miniconda/envs/dles/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/agostini/miniconda/envs/dles/lib/python3.7/runpy.py", line 85, in _run_code
    exec

ValueError: Cannot use 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/Relu' as input to 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/act_quant/FakeQuantWithMinMaxVars' because 'Tacotron_model/inference/decoder/while/CustomDecoderStep/decoder_prenet/dense_2/Relu' is in a while loop. See info log for more details.